# Recommender baseline SVD+KNN


Содержание:
1. [Описание подхода](#Описание-подхода)
2. [Подготовка данных](#Подготовка-данных)
3. [Обучение модели](#Обучение-модели)
4. [Тестирование модели](#Тестирование-модели)

### Описание подхода
Во время обучения строится разряженная матрица товар-чек. К данной матрице применяется алгоритм матричного разложения SVD.
С помощью матричнго разложения SVD для каждого товара из ассортимента вычисляется эмбеддинг. Затем на товары из текущей корзины клиента применяем обученный SVD, получаем их эмбеддинги и находим ближайшие к ним товары из ассортимента по косинусной мере.

In [271]:
%enable_full_walk


import numpy as np
import pandas as pd
import scipy


import os
import sys
import functools as ft
sys.path.append(ft.reduce(os.path.join, ['..']))

import pickle
import time

from src.svdrec import RecommenderSVDKNN, mean_precision

from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split

def load_mapping(filename):
    map_path = ft.reduce(os.path.join, ['..','data','mapping' ,filename+'.pkl'])
    with open(map_path, 'rb') as handle:
        return pickle.load(handle)

### Подготовка данных

In [272]:
product_id_to_idx = load_mapping('product_id_to_idx')

# Загрузка датасета со всеми транзакциями. Датасет вида: время-транзакция-товар
path = ft.reduce(os.path.join, ['..','data', 'processed', 'purchases.csv'])
df = pd.read_csv(path, dtype={'transaction_idx': str, 'product_idx': int})
df = df[['transaction_datetime', 'transaction_idx', 'product_idx']]

#Датасеты для обучения и тестирования имеют разные форматы, поэтому разделяем их по врмени.
split_date = '2019-03-12 15:00:00' 
train_transactions = df[df.transaction_datetime < split_date]
del df

# Загружаем датасет для тестирования. Для оценки всех моделей используется единый датасет.
cols = ['client_idx']+['b_' + str(x) for x in range(len(product_id_to_idx))]+['target_product', 'target']
X_test = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'client_hist_context_target_product.csv']), usecols=cols)
X_test = X_test[X_test.target == 1]
X_test.set_index('client_idx', inplace=True)

_, X_test, _, y_test= \
    train_test_split(X_test.drop('target', axis=1), X_test.target, test_size=0.2, shuffle=False)

# Создаем разреженную матрицу чек-товар
rows = train_transactions.transaction_idx.astype(int)
cols = train_transactions.product_idx.astype(int)
sp_train = scipy.sparse.coo_matrix((np.ones_like(rows), (rows, cols)))
del rows
del cols

### Обучение модели

In [273]:
# Обучаем модель
model = RecommenderSVDKNN(emb_dim=500)
model.fit(sp_train.T)

### Тестирование модели

In [274]:
test_input = []
time_distr = []
target_input = X_test['target_product'].values

for index, user in X_test.iterrows():
    
    start = time.time()
    
    # Переводим товары в корзине в нормальный id
    basket = user['b_0':'b_199']
    basket.reset_index(inplace=True, drop=True)
    test_pr = basket[basket.values].index.values
    
    # Делаем прогноз
    pred = model.predict_sample(test_pr, num=3)
    
    # Подсчет времени выполнения и сохранение результата
    end = time.time()
    time_distr.append(end - start)
    test_input.append(list(pred))
    

In [275]:
# Расчет метрики
print('Mean Precision:', mean_precision(target_input, test_input))

Mean Precision: 0.25902322319844906
